### Notebook for finding categories that our teams excel at, so that each team gets an award

In [ ]:
import pandas as pd

In [ ]:
import sys
sys.path.append("..")
from base_python_files.api_info import *
from base_python_files.credentials import *
from base_python_files.api_functions import *
from base_python_files.stat_functions import *

In [ ]:
d = retrieve_espn_data(swid,espn_s2,league_id)
data = clean_data(d)

In [ ]:
skater_cats = [
    "FOW",
    "assists",
    "average_time_on_ice",
    "blocks",
    "goals",
    "hits",
    "penalty_minutes",
    "plus_minus",
    "power play points",
    "shots_on_goal",
]

In [ ]:
possible_max_categories = skater_cats+['short handed goals','short handed assists','game winning goals','special team points','clean_sheets','shots_at_them','saves','over_time_loss','FOPercent']
possible_min_categories = ['goals against average','goals against']

award_df = data[~data["LeagueTeamID"].isna()].copy()

In [ ]:
for cat in possible_max_categories:
    try:
        award_df['test'] = award_df[cat+'AT']-award_df[cat+'proj']
        top = award_df[["Name","LeagueTeamID",cat+'AT',cat+'proj',"games_playedAT",'test']].sort_values(by='test',ascending=False)
        if (7 in list(top.iloc[:4]['LeagueTeamID'])) :
            print()
            print(cat)
            print(top.iloc[:4])
    except KeyError:
        print(f"No columns: {cat}")

In [ ]:
award_df.sort_values(by="hatricksAT",ascending=False)[["Name","LeagueTeamID","hatricksAT","games_playedAT"]]

In [ ]:
birth_dates_df = pd.read_csv("player_bios.csv",header=0)
birth_dates_df["Age"]=pd.to_datetime('today').year-pd.to_datetime(birth_dates_df["DOB"]).dt.year
birth_dates_df

In [ ]:
for i in range(1,9):
    award_team_df= award_df[award_df["LeagueTeamID"]==i]
    ages = birth_dates_df[birth_dates_df["Player"].isin(list(award_team_df["Name"]))]["Age"]
    print(i,ages.sum()/len(ages))

In [ ]:
league_team_awards ={k:{} for k in range(1,9)}

for cat in possible_max_categories:
    if cat not in ["games_played"] and cat+"AT" in award_df.columns:
        
        cat_term = cat+"AT"
        if cat =="FOPercent":
            cat_df =award_df[award_df["FOWAT"]>10][["Name","LeagueTeamID",cat_term,"games_playedAT"]].copy()
        elif cat=="average_time_on_ice":
            cat_df=award_df[award_df["Position"]!="GK"][["Name","LeagueTeamID",cat_term,"games_playedAT"]].copy()
        else:
            cat_df = award_df[["Name","LeagueTeamID",cat_term,"games_playedAT"]].copy()
        cat_df[cat_term]=cat_df[cat_term]*cat_df["games_playedAT"]
        top = cat_df.sort_values(by=cat_term,ascending=False).iloc[0:4]
        if top[cat_term].iloc[0]==top[cat_term].iloc[1]:
            print()
            print("DRAW")
            print(cat_term)
            print(top)
        else:
            league_team_awards[int(top.iloc[0]["LeagueTeamID"])][cat]=top.iloc[0]

In [ ]:
league_team_awards

In [ ]:
birth_places = pd.read_csv("birthplaces_BC.csv",header=1)
birth_places

In [ ]:
for i in range(1,9):
    award_team_df = award_df[award_df["LeagueTeamID"]==i]
    print(i,len(award_team_df[award_team_df["Name"].isin(birth_places["Player"])]))

In [ ]:
award_df['test']=award_df['shots_on_goalAT']+award_df['hitsAT']+award_df['blocksAT']
award_df.sort_values(by='test',ascending=False)[['Name','test']]